# ML-MOGA User Manual

Languages: Python(3.7.9), C++ <br />
Tools: PyTorch(1.4.0+), Jupyter Notebook(6.2.0) <br />
CAM(cam.lbl.gov): Data processing and training <br />
NERSC: MOGA running(both Tr-MOGA & ML-MOGA)

If you have any question, contact yupinglu89@gmail.com

## Tr-MOGA 

#### 1. Load modules on NERSC

In [ ]:
module purge
module load PrgEnv-gnu
module load openmpi
module load gsl
module load pytorch/v1.6.0

#### 2. Set up MOGA code

Example code(nl-run/run_ori/1_moga). <br />
Compile the code in ML_package folder and get the executable by typing "make".<br />
Submit scanjob.s to schedule the job on NERSC. The time limit on NERSC is 48 hrs for regular queue. You can also submit to 30 minutes debug queue for fast execution.

In [ ]:
#!/bin/bash
#SBATCH --qos=regular
#SBATCH --time=48:00:00
#SBATCH --nodes=64
#SBATCH --tasks-per-node=32
#SBATCH --constraint=haswell
#SBATCH --mail-user=yupinglu89@gmail.com
#SBATCH --mail-type=ALL
#SBATCH --job-name=MOGA

cd $SLURM_SUBMIT_DIR
echo Working directory is : $SLURM_SUBMIT_DIR

echo $SLURM_JOB_NODELIST
echo $SLURM_JOBID
echo $SLURM_NPROCS

# NCPUS=`wc -l $SLURM_NODELIST| awk '{print $1}'`
# JobID=`echo ${SLURM_JOBID} | cut -f1 -d.`

NCPUS=$SLURM_NPROCS
JobID=$SLURM_JOBID

mkdir Dir_$JobID
cd Dir_$JobID
cp ../problem.cpp ./
cp ../tracking.in ./
cp ../scanjob.s ./
cp ../ALSU.cpp ./
cp ../ALSU.h ./
cp ../input_gen.dat ./

echo "Start parallel job with CPUS"
echo $NCPUS
echo " -----------------------------------------------"

#module purge
#module load PrgEnv-gnu
#module load openmpi
#module load gsl
#module load pytorch
####### Problem mode (0-DASearch, 1-FreqMap, 2-DiffMomen, 3-AreaMomen) ###########
#####  1 for read pop, 2 for read gen
EXEC="../nsga2r 0.5 3  2"

mpirun -v -np $NCPUS $EXEC <../tracking.in >$SLURM_SUBMIT_DIR/stdout_$JobID.out 2>$SLURM_SUBMIT_DIR/stderr_$JobID.out

mv ../*$JobID.out ../slurm-*.out ../Dir_$JobID/

### END of job
echo "Job complete"

#### 3. Retrieve results (gen_*_db.dat files)

These files are stored in Dir_* folder.<br />
Delete the first line of each gen_*_db.dat file.

     20 outputs,   11 variables,   conViol  rank  crowDist

#### 4. Change random seeds

We tested the moga code by changing two random seeds.<br />
Change MOGA random seed: EXEC="../nsga2r 0.1 3  2" (in scanjob.s)<br />
Change lattice error random seed: srand(2021); (in problem.cpp)

## Machine Learning Approach

+ We first preprocess training data acquired from prior simulations and use this data to obtain two well-trained models using the neural network (NN) depicted below. 
+ We then use these two NN models to replace DA/MA particle tracking in MOGA while the rest of the MOGA setup remains the same as in the original tracking-based MOGA (Tr-MOGA). 
+ We evaluate the results.

<img src="1.png" alt="drawing" width="800"/>

8-layer fully-connected (FC) NN architecture for DA and MA prediction.

## ML-MOGA 



#### 1. Training Data

We used the first 10 dat files as training data. These data are stored in dat folder on CAM. Below are two python scripts to preprocess the data (include filtering out those not meet the constraints).

python pre.da.py<br />
python pre.ma.py

In [ ]:
#!/usr/bin/env python
# coding: utf-8
'''
MOGA data preprocessing for dynamic aperture
pre.da.py
'''

# load libs
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle

# load files
path = './dat/'
fs = os.listdir(path)
data = []

# read files
for f in fs:
    tmp_df = pd.read_csv(path+f, header=None)
    data.append(tmp_df)
df = pd.concat(data, ignore_index=True, sort =False)

# get X and Y
data = df.to_numpy()
x = data[:, -3]
data = data[x == 0]
X = data[:,20:31]
Y_t = data[:,15:17]
Y = np.mean(Y_t, axis=1)

# split data into training set and test set
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(X, Y, test_size=0.20, random_state=2020)

# data normalization to [0, 1]
x_mean = np.mean(x_train_o, axis=0)
x_std = np.std(x_train_o, axis=0)
print(x_mean)
print(x_std)

x_train = (x_train_o - x_mean) / x_std
x_test = (x_test_o - x_mean) / x_std
y_train = y_train_o
y_test = y_test_o 
print(len(y_test))

moga = {
    "x_train": x_train,
    "x_test": x_test,
    "y_train": y_train,
    "y_test": y_test,
    "x_mean": x_mean,
    "x_std": x_std,
}

# save data
pickle.dump(moga, open("da.1208.pkl", "wb" ))

In [ ]:
#!/usr/bin/env python
# coding: utf-8
'''
MOGA data preprocessing for momentum aperture
pre.ma.py
'''

# load libs
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle

# load files
path = './dat/'
fs = os.listdir(path)
data = []

# read files
for f in fs:
    tmp_df = pd.read_csv(path+f, header=None)
    data.append(tmp_df)
df = pd.concat(data, ignore_index=True, sort =False)

# get X and Y
data = df.to_numpy()
x = data[:, -3]
data = data[x == 0]
X = data[:,20:31]
Y = data[:,17]

# split data into training set and test set
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(X, Y, test_size=0.20, random_state=2020)

# data normalization to [0, 1]
x_mean = np.mean(x_train_o, axis=0)
x_std = np.std(x_train_o, axis=0)
print(x_mean)
print(x_std)

x_train = (x_train_o - x_mean) / x_std
x_test = (x_test_o - x_mean) / x_std
y_train = y_train_o
y_test = y_test_o 
print(len(y_test))

moga = {
    "x_train": x_train,
    "x_test": x_test,
    "y_train": y_train,
    "y_test": y_test,
    "x_mean": x_mean,
    "x_std": x_std,
}

# save data
pickle.dump(moga, open("ma.1208.pkl", "wb" ))

#### 2. Model Training

The two model scripts will load ma.1208.pkl and da.1208.pkl and start the model training.

nohup python ma.py > ma.out 2> ma.err < /dev/null & <br />
nohup python da.py > da.out 2> da.err < /dev/null &

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# ma.py
# load libs
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle

# definition of a simple fc model [11, 32, 64, 1]
class MOGANet(nn.Module):
    def __init__(self):
        super(MOGANet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(11, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x

class MOGADataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
    
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

# define train and test functions
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        # compute output
        outputs = model(x)
        loss = criterion(outputs, y)
        train_loss += loss.item()
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train_loader)
    return train_loss

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            
            # compute output
            outputs = model(x)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            
    test_loss /= len(test_loader)
    return test_loss

# Use CUDA
device = torch.device("cpu")

random.seed(2020) 
torch.manual_seed(2020)
torch.cuda.manual_seed_all(2020)

# load files
moga = pickle.load(open("ma.1208.pkl", "rb"))
x_train, x_test = moga["x_train"], moga["x_test"]
y_train, y_test = moga["y_train"], moga["y_test"]
x_mean, x_std = moga["x_mean"], moga["x_std"]

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train.reshape(-1,1)).float()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test.reshape(-1,1)).float()

# Parameters
params = {'batch_size': 128,
          'shuffle': True,
          'num_workers': 4}

# dataloader
train_data = MOGADataset(x_train, y_train)
train_loader = DataLoader(dataset=train_data, **params)

test_data = MOGADataset(x_test, y_test)
test_loader = DataLoader(dataset=test_data, **params)

model = MOGANet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8)

# Training here
for t in range(500):
    train_loss = train(model, device, train_loader, optimizer, criterion) 
    test_loss = test(model, device, test_loader, criterion)
    if t%10 == 0:
        print(t, train_loss, test_loss)
    
    scheduler.step(test_loss)

# save trained model
torch.save(model.state_dict(), 'ma.1208.pth')
#model.load_state_dict(torch.load("ma.1208.pth"))

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# da.py 
# load libs
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle

# definition of a simple fc model [11, 32, 64, 1]
class MOGANet(nn.Module):
    def __init__(self):
        super(MOGANet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(11, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x

class MOGADataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
    
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

# define train and test functions
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        # compute output
        outputs = model(x)
        loss = criterion(outputs, y)
        train_loss += loss.item()
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train_loader)
    return train_loss

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            
            # compute output
            outputs = model(x)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            
    test_loss /= len(test_loader)
    return test_loss

# Use CUDA
device = torch.device("cpu")

random.seed(2020) 
torch.manual_seed(2020)
torch.cuda.manual_seed_all(2020)

# load files
moga = pickle.load(open("da.1208.pkl", "rb"))
x_train, x_test = moga["x_train"], moga["x_test"]
y_train, y_test = moga["y_train"], moga["y_test"]
x_mean, x_std = moga["x_mean"], moga["x_std"]

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train.reshape(-1,1)).float()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test.reshape(-1,1)).float()

# Parameters
params = {'batch_size': 128,
          'shuffle': True,
          'num_workers': 4}

# dataloader
train_data = MOGADataset(x_train, y_train)
train_loader = DataLoader(dataset=train_data, **params)

test_data = MOGADataset(x_test, y_test)
test_loader = DataLoader(dataset=test_data, **params)

model = MOGANet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8)

# Training here
for t in range(500):
    train_loss = train(model, device, train_loader, optimizer, criterion) 
    test_loss = test(model, device, test_loader, criterion)
    if t%10 == 0:
        print(t, train_loss, test_loss)
    
    scheduler.step(test_loss)

# save trained model
torch.save(model.state_dict(), 'da.1208.pth')
#model.load_state_dict(torch.load("da.1208.pth"))

#### 3. Format converting

We need to convert two generated model files ending with .pth to .pt format. The new files da.1208.pt and ma.1208.pt will be inserted into MOGA code to start ML-MOGA run. We first need to provide the current mean and std of our training data to ts.da.py and ts.ma.py before the conversion.

python ts.da.py <br />
python ts.ma.py

#### 4. ML-MOGA run

1) Copy the converted model files (da.1208.pt and ma.1208.pt) to ML_package.

2) Copy libtorch library files to ML_package.

3) Replace problem.cpp with PyTorch models. The ML version of problem.cpp is listed below

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <unistd.h>
#include "Tracy.Meso.h"
#include "ALSU.h"
#include "global.h"
#include <torch/script.h>
#include <iostream>
#include <stdexcept>
#include <string>

extern ALS_U_V17 *SR;

string exec(string command) {
   char buffer[128];
   string result = "";

   // Open pipe to file
   FILE* pipe = popen(command.c_str(), "r");
   if (!pipe) {
      return "popen failed!";
   }

   // read till end of process:
   while (!feof(pipe)) {

      // use buffer to read and add to result
      if (fgets(buffer, 128, pipe) != NULL)
         result += buffer;
   }

   pclose(pipe);
   return result;
}

double SegQ, SegB;
ALS_U_V17 *Ring() {
    SegQ = 1;
    SegB = 3;
    SR = new ALS_U_V17(SegQ,SegB);

    SR->Individualize();
    SR->SetIntegrator(4);
    SR->SetSec6(10);

    return SR;
}

void test_problem (int gen, int nload, double *xvar,  double *obj, double *constr,double *db) {
    int i,j;
    double maxBx=0,maxBy=0,maxEta=0;
    double B3Betax = 0,B3Betay=0;
    Belement *C;

    SR->clearQuadGradErr();
    SR->clearQuadSkew();
    SR->ClearRef();
    SR->ClearCOD();
    SR->FixedPoint.clear();
    SR->FixedPoint6.clear();
    SR->ClearRef6();
    SR->SetdP(0.0);
    SR->Sextpoles_TurnOff();

    double kqf1 =  xvar[0];
    double kqf2 =  xvar[1];
    double kqf3 =  xvar[2];
    double kqf4 =  xvar[3];
    double kqf5 =  xvar[4];
    double kqf6 =  xvar[5];
    double kqd1 =  xvar[6];
    double kb1 =  xvar[7];
    double kb2 =  xvar[8];
    double kb3 =  xvar[8];
    double kshh, kshh2; 
    // Input for training mdoel
    kshh = xvar[9];
    kshh2 = xvar[10];

    ////////////////////////                                                                                                                                   
    // linear lattice                                                                                                                                          
    ////////////////////                                                                                                                                       

    SR->setKQf1(kqf1);
    SR->setKQf2(kqf2);
    SR->setKQf3(kqf3);
    SR->setKQf4(kqf4);
    SR->setKQf5(kqf5);
    SR->setKQf6(kqf6);
    SR->setKQd1(kqd1);
    SR->Quads[7]->SetK(kb1);
    SR->Quads[8]->SetK(kb2);
    SR->Quads[9]->SetK(kb3);

#ifdef TESTRUN
    SR->GetTwiss(1);
#endif

    if(!SR->GetTwiss(1))
    {
        for (i=0;i<ndb;i++)     db[i]=999.0;
        for (i=0;i<ncon;i++)    constr[i]=-1.0;
        for (i=0;i<nobj;i++)    obj[i] = 999.0+i*fabs(xvar[0]);
        return;
    }

    SR->CalcIntegral();
    SR->CalcEmittance(2.0E9);
    SR->SetA44();
  
#ifdef TESTRUN

    SR->GetTwiss(1);
    SR->CalcIntegral();
    SR->ListSynch(stdout);
    SR->listTwissTable("Twiss.txt");

    C=&(SR->Belem[0]);
    FILE *fid = fopen("magnet.txt","w");
    for(int n=0; n< SR->NoB;n++) {
        C = &(SR->Belem[n]);
        fprintf(fid," %d   %s  %d   %d  \n", n, C->Elem->Name.c_str(), C->Elem->GetSec6(), C->Elem->IntegMethod );

    }
    fclose(fid);
#endif

    C=&(SR->Belem[0]);

    db[0] = C->TwissH.v[1]; //betax at s=0;                                                                                                                  
    db[1] = C->TwissV.v[1]; //betay at s = 0;                                                                                                                
    db[10] = C->Eta.v[0];   //etax at s= 0;                                                                                                                  

    int inj_sn,qf1_sn,qf2_sn,qf3_sn,qf4_sn,qf5_sn,qf6_sn;

    for(int n=0; n< SR->NoB;n++) {
        C = &(SR->Belem[n]);
        if (C->TwissH.v[1]>maxBx) maxBx = C->TwissH.v[1];
        if (C->TwissV.v[1]>maxBy) maxBy = C->TwissV.v[1];
        if (C->Eta.v[0]>maxEta) maxEta = C->Eta.v[0];

        if (strncmp("B3M",C->Elem->Name.c_str(),3)==0) {
            B3Betax = C->TwissH.v[1];
            B3Betay = C->TwissV.v[1];
        }

        if(strncmp("SECT1",C->Elem->Name.c_str(),5)==0) inj_sn = 0;
        if(strncmp("QF1(01)",C->Elem->Name.c_str(),7)==0) qf1_sn = n;
        if(strncmp("QF2(01)",C->Elem->Name.c_str(),7)==0) qf2_sn = n;
        if(strncmp("QF6(01)",C->Elem->Name.c_str(),7)==0) qf6_sn = n;
    }                                                                                                

    db[2] = SR->NtlEmittance;  //emittance                                                                                                    
    db[3] = SR->A44.m[0][0]+SR->A44.m[1][1]; //tracex                                                                                                          
    db[4] = SR->A44.m[2][2]+SR->A44.m[3][3]; //tracey                                                                                                          
    db[5] = SR->Jx;  //jx                                                                                                                       
    db[6] = SR->Jz;  //jy                                                                                                                     
    db[7] = SR->Je;  //j                                                                                                                                      
    db[8] = SR->TuneH;   //tunex                                                                                                                                 
    db[9] = SR->TuneV;   //tuney     
    db[11] = SR->ChromH;
    db[12] = SR->ChromV;
    db[13] = B3Betax;
    db[14] = B3Betay;

    constr[0] = 2-fabs(db[3]);
    constr[1] = 2-fabs(db[4]);
    constr[2] = db[5];
    constr[3] = db[6];
    constr[4] = db[7];
    constr[5] = 30-maxBx;  // maximum betax <25 m                                                                                                                
    constr[6] = 30-maxBy; // maximum betay <25 m                                                                                                                 
    constr[7] = 0.15-maxEta; // maximum etax <15 cm                                                                                                              
    constr[8] = (155e-12-db[2]);  // emit <150 pm-rad 

    double ftunex = fabs(db[8]-(int) db[8]);
    double ftuney = fabs(db[9]-(int) db[9]);

    constr[9] = (ftunex-0.1)*(0.4-ftunex);
    constr[10] = (ftuney-0.1)*(0.4-ftuney);
    constr[11] = 1e-3-fabs(db[10]); //etax <1mm at s = 0;                                                                                                        
    constr[12] = (db[0]-1)*(5-db[0]); //1<betax<5
    constr[13] = (db[1]-1)*(5-db[1]); //1<betay<5
    constr[14] = (4-B3Betax)*(4-B3Betay);
    constr[15] = 0.01-fabs(ftunex-ftuney);

    /* normialize the constraints*/
    for (i=0;i<16;i++) {
        if (constr[i]<0||isnan(constr[i])) {
#ifdef TESTRUN
            printf(" ----- constr vio %d, %f\n",i,constr[i]);
#endif 
            constr[i]=-1;
        }
    }
    
    obj[0]=db[2];
    constr[16] = 0.0;
    constr[17] = 0.0;

    //////////////////////////                                                                                                                          
    // nonliear optimziation                                                                                                                                   
    //////////////////////////   

    SR->Sexts[2]->SetK(kshh);
    SR->Sexts[3]->SetK(kshh2);

    if (!SR->FitChrom(1.0,1.0)) {
        db[15]=0.0;
        db[16]=0.0;
        db[17]= 0.0;

        db[18]=0.0;
        db[19]= 0.0;
        constr[16] = -1.0;
        constr[17] = -1.0;
        obj[1]=9999.0;

        return;
    }

    db[18] = SR->GetSF();
    db[19] = SR->GetSD();

    constr[16] = 900-fabs(db[18]);
    constr[17] = 900-fabs(db[19]);  

    srand(199);
    SR->setQuadSkew(5e-4);
    SR->setQuadGradErr(2e-4);
  
#ifdef TESTRUN
    printf(" ---SF: %32.28f, SD:%32.28f\n",SR->GetSF(),SR->GetSD());
    SR->GetTwiss(1);
    SR->CalcIntegral();
    SR->ListSynch(stdout);
    SR->listTwissTable("Twiss_1.txt");
    SR->Tracy2AT("lattice.m");
#endif
  
    //double MomAper1[2],MomAper2[2],MomAper3[2],MomAper4[2];

#ifdef TESTRUN
    
#else
    // Input
    float aa = (xvar[0] - 14.028049) / 3.80621681e-01;
    float bb = (xvar[1] - 9.7394512) / 5.89510194e-01;
    float cc = (xvar[2] - 11.783254) / 9.37556664e-01;
    float dd = (xvar[3] - 15.524115) / 2.50345962e-01;
    float ee = (xvar[4] - 15.713844) / 2.14431638e-01;
    float ff = (xvar[5] - 15.480341) / 3.52844788e-01;
    float gg = (xvar[6] - -14.873348) / 3.24311876e-01;
    float hh = (xvar[7] - -2.3504979) / 1.64542081e-01;
    float ii = (xvar[8] - -7.1688257) / 2.05577563e-01;
    float jj = (xvar[9] - -75.974958) / 2.79143321e+02;
    float kk = (xvar[10] - -157.57141) / 3.02909397e+02;

    auto X = torch::tensor({aa, bb, cc, dd, ee, ff, gg, hh, ii, jj, kk}, torch::requires_grad(false).dtype(torch::kFloat32)).view({1,11});
    vector<torch::jit::IValue> inputs;
    inputs.push_back(X);

    // da
    torch::jit::script::Module module_da = torch::jit::load("da.1208.pt");

    // Execute the model and turn its output into a tensor.
    at::Tensor output_da = module_da.forward(inputs).toTensor();
    db[15] = output_da.item<float>();
    db[16] = output_da.item<float>();
    
    // ma
    torch::jit::script::Module module_ma = torch::jit::load("ma.1208.pt");
    
    // Execute the model and turn its output into a tensor.
    at::Tensor output_ma = module_ma.forward(inputs).toTensor();
    db[17] = output_ma.item<float>();
#endif

    obj[1]=(db[15]+db[16])/2;
    obj[2]=db[17];

    if (isnan(obj[1])) obj[1]=9999.0;
    if (isnan(obj[2])) obj[2]=9999.0; 
}

4) Remember to replace float aa - kk with the current mean and std, same as format conversion.

5) Other minor changes please refer to NERSC/ml-run/7_moga/ML_package.

6) Create build directory and run the following command to compile executable.


In [ ]:
make build

cd build

cmake -DCMAKE_CXX_COMPILER=mpiCC -DCMAKE_C_COMPILER=mpicc -DCMAKE_PREFIX_PATH=./libtorch ..

make

mv nsga2r ..

7) Submit the job script

In [ ]:
sbatch scanjob.s